In [456]:
import cv2
import numpy as np

## define general function: MSE and show_image

In [457]:
def MSE(img1, img2):
    return np.sum((img1-img2)**2)/img1.shape[0]/img1.shape[1]/img1.shape[2]
def show_image(img, name):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [461]:
def re_construct_img(new_img, original_img):
    res = np.zeros(original_img.shape, dtype=np.uint8)
    colors = []
    for bin in new_img:
        color = bin[0]
        #print(color[0:3])
        colors.append(color[0:3])
        for pixel in bin:
            if color[0]!=pixel[0] or color[1]!=pixel[1] or color[2]!=pixel[2]:
                raise Exception("fuck up")
            res[pixel[3], pixel[4]] = color[0:3]
    return res, colors
def generate_indexed_img(img):
    index_i_j = 2
    img_with_index = np.zeros([img.shape[0],img.shape[1], img.shape[2]+index_i_j], dtype=int)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            u = np.zeros([img.shape[2]+index_i_j], dtype=int)
            u[0] = img[i,j,0]
            u[1] = img[i,j,1]
            u[2] = img[i,j,2]
            u[3] = i
            u[4] = j
            img_with_index[i,j] = u
    return img_with_index.reshape(-1, img_with_index.shape[2])
def median_cut_quantization(img,current_level, n, bins):
    #print('here',img.shape)
    if current_level == n:
        #find the mean of the img
        rmean = np.mean(img[:,0])
        gmean = np.mean(img[:,1])
        bmean = np.mean(img[:,2])
        #print('in max level ',img.shape, ' quantized color: ',rmean,gmean,bmean)
        #print(img)
        img[:,0] = rmean
        img[:,1] = gmean
        img[:,2] = bmean
        #print(img)
        
        bins.append(img)
        return
    #find the longest side of the img
    def find_widest_side(img):
        r_gap = np.max(img[:,0]) - np.min(img[:,0])
        g_gap = np.max(img[:,1]) - np.min(img[:,1])
        b_gap = np.max(img[:,2]) - np.min(img[:,2])
        if r_gap >=g_gap and r_gap>=b_gap:
            return 0
        elif g_gap >=r_gap and g_gap>=b_gap:
            return 1
        else:
            return 2
    widest_side_color = find_widest_side(img)
    
    #find the median
    widest_side = img[:,widest_side_color]
    sorted_widest_side = np.sort(widest_side)
    median = sorted_widest_side[int(sorted_widest_side.shape[0]/2)]
    
    #split the img into two parts
    cub_a = img[img[:,widest_side_color]<=median]
    cub_b = img[img[:,widest_side_color]>median]
    #recursive call the function
    median_cut_quantization(cub_a,current_level+1,n,bins)
    median_cut_quantization(cub_b,current_level+1,n,bins)
def ordered_dither(img, dither_matrix):
    res = np.zeros(img.shape, dtype=np.uint8)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            for k in range(img.shape[2]):
                if img[i,j,k] > dither_matrix[i%dither_matrix.shape[0],j%dither_matrix.shape[1]]:
                    res[i,j,k] = 255
                else:
                    res[i,j,k] = 0
    return res

In [464]:
n = 3
target_image = '.\\img\\Lenna.jpg'
img = cv2.imread(target_image, cv2.IMREAD_COLOR)
indexed_image = generate_indexed_img(img)

In [465]:
bins = []
median_cut_quantization(indexed_image,0,n,bins)
median_cut_quantization_img, colors = re_construct_img(bins,img)
print(colors)
show_image(median_cut_quantization_img, 'median cut quantization')
print('mse ' +'in n = '+str(n)+' after the image are quantized by median cut: {}'.format(MSE(img, median_cut_quantization_img)))
print(median_cut_quantization_img)

[array([62, 24, 99]), array([ 89,  48, 116]), array([ 78,  67, 175]), array([ 95,  88, 191]), array([119, 117, 193]), array([107, 117, 222]), array([127, 151, 225]), array([169, 187, 226])]
mse in n = 3 after the image are quantized by median cut: 66.94910671366769
[[[127 151 225]
  [127 151 225]
  [127 151 225]
  ...
  [127 151 225]
  [127 151 225]
  [119 117 193]]

 [[127 151 225]
  [127 151 225]
  [127 151 225]
  ...
  [127 151 225]
  [127 151 225]
  [119 117 193]]

 [[127 151 225]
  [127 151 225]
  [127 151 225]
  ...
  [127 151 225]
  [127 151 225]
  [119 117 193]]

 ...

 [[ 62  24  99]
  [ 62  24  99]
  [ 62  24  99]
  ...
  [ 78  67 175]
  [ 78  67 175]
  [ 78  67 175]]

 [[ 62  24  99]
  [ 62  24  99]
  [ 62  24  99]
  ...
  [ 78  67 175]
  [ 78  67 175]
  [ 78  67 175]]

 [[ 62  24  99]
  [ 62  24  99]
  [ 62  24  99]
  ...
  [ 78  67 175]
  [ 78  67 175]
  [ 78  67 175]]]


In [466]:
cv2.imwrite('.\\out\\'+'Lenna_'+str(n)+'.jpg', median_cut_quantization_img)
target_image = '.\\out\\'+'Lenna_'+str(n)+'.jpg'
img = cv2.imread(target_image, cv2.IMREAD_COLOR)
print(img)

[[[129 150 225]
  [128 148 225]
  [131 151 229]
  ...
  [127 150 225]
  [127 151 223]
  [ 99 122 198]]

 [[127 147 225]
  [133 153 231]
  [130 149 230]
  ...
  [130 147 233]
  [129 144 223]
  [108 120 192]]

 [[131 149 232]
  [126 143 229]
  [126 143 230]
  ...
  [137 145 234]
  [145 146 227]
  [128 120 190]]

 ...

 [[ 60  26  96]
  [ 60  22  94]
  [ 65  23 100]
  ...
  [ 87  63 158]
  [ 87  70 173]
  [ 80  67 175]]

 [[ 60  25  99]
  [ 63  25  97]
  [ 64  24  96]
  ...
  [ 86  71 176]
  [ 81  70 180]
  [ 73  65 179]]

 [[ 59  23 100]
  [ 65  27  99]
  [ 62  25  95]
  ...
  [ 78  69 180]
  [ 72  65 180]
  [ 71  64 179]]]


In [467]:
dither_matrix = np.array([
    [  0, 192,  48, 240],
    [128,  64, 176, 112],
    [ 32, 224,  16, 208],
    [160,  96, 144,  80]
], dtype=np.uint8)
median_cut_quantization_with_dither_img = ordered_dither(median_cut_quantization_img, dither_matrix)
show_image(median_cut_quantization_with_dither_img, 'ordered_dither')
print('mse ' +'in n = '+str(n)+' after the image are quantized by median cut with ordered dither: {}'.format(MSE(img, median_cut_quantization_with_dither_img)))

mse in n = 3 after the image are quantized by median cut with ordered dither: 105.86343334401539


In [453]:
print(median_cut_quantization_img)
print(median_cut_quantization_with_dither_img)

[[[127 151 225]
  [127 151 225]
  [127 151 225]
  ...
  [127 151 225]
  [127 151 225]
  [119 117 193]]

 [[127 151 225]
  [127 151 225]
  [127 151 225]
  ...
  [127 151 225]
  [127 151 225]
  [119 117 193]]

 [[127 151 225]
  [127 151 225]
  [127 151 225]
  ...
  [127 151 225]
  [127 151 225]
  [119 117 193]]

 ...

 [[ 62  24  99]
  [ 62  24  99]
  [ 62  24  99]
  ...
  [ 78  67 175]
  [ 78  67 175]
  [ 78  67 175]]

 [[ 62  24  99]
  [ 62  24  99]
  [ 62  24  99]
  ...
  [ 78  67 175]
  [ 78  67 175]
  [ 78  67 175]]

 [[ 62  24  99]
  [ 62  24  99]
  [ 62  24  99]
  ...
  [ 78  67 175]
  [ 78  67 175]
  [ 78  67 175]]]
[[[255 255 255]
  [  0   0 255]
  [255 255 255]
  ...
  [  0   0 255]
  [255 255 255]
  [  0   0   0]]

 [[  0 255 255]
  [255 255 255]
  [  0   0 255]
  ...
  [255 255 255]
  [  0   0 255]
  [255 255 255]]

 [[255 255 255]
  [  0   0 255]
  [255 255 255]
  ...
  [  0   0 255]
  [255 255 255]
  [  0   0   0]]

 ...

 [[  0   0   0]
  [  0   0 255]
  [  0   0   0]
  ..

In [474]:
cv2.imwrite('haha.jpg', median_cut_quantization_with_dither_img)*255
target_image = 'haha.jpg'
img = cv2.imread(target_image, cv2.IMREAD_COLOR)
print(img)
#from PIL import Image
#
#PIL_image = Image.fromarray(np.uint8(median_cut_quantization_with_dither_img)).convert('RGB')
#PIL_image.save("your_file.png")
#img = Image.open("your_file.png")
##img.show()
#print(median_cut_quantization_with_dither_img)

[[[211 235 255]
  [ 14  72 108]
  [179 255 255]
  ...
  [ 19  50 153]
  [227 239 255]
  [  0  11   0]]

 [[164 219 255]
  [197 234 255]
  [ 23  45 151]
  ...
  [177 224 255]
  [ 44  48 150]
  [255 227 255]]

 [[179 240 255]
  [ 19  42 164]
  [216 208 255]
  ...
  [ 12  53 156]
  [239 210 255]
  [ 30   0 111]]

 ...

 [[  0   0  50]
  [ 97  44 124]
  [ 40   0  49]
  ...
  [250 255 223]
  [  0   0  28]
  [ 59  34 162]]

 [[133  56 183]
  [ 13   0  61]
  [247 237 255]
  ...
  [  0   0  49]
  [236 225 255]
  [  0   0  77]]

 [[ 30   0  93]
  [ 65  42 140]
  [  0   0  50]
  ...
  [ 46   1 228]
  [ 19  19 209]
  [  0  40 171]]]


In [475]:
print('========')
wtf = np.array(img)
print(wtf)

[[[211 235 255]
  [ 14  72 108]
  [179 255 255]
  ...
  [ 19  50 153]
  [227 239 255]
  [  0  11   0]]

 [[164 219 255]
  [197 234 255]
  [ 23  45 151]
  ...
  [177 224 255]
  [ 44  48 150]
  [255 227 255]]

 [[179 240 255]
  [ 19  42 164]
  [216 208 255]
  ...
  [ 12  53 156]
  [239 210 255]
  [ 30   0 111]]

 ...

 [[  0   0  50]
  [ 97  44 124]
  [ 40   0  49]
  ...
  [250 255 223]
  [  0   0  28]
  [ 59  34 162]]

 [[133  56 183]
  [ 13   0  61]
  [247 237 255]
  ...
  [  0   0  49]
  [236 225 255]
  [  0   0  77]]

 [[ 30   0  93]
  [ 65  42 140]
  [  0   0  50]
  ...
  [ 46   1 228]
  [ 19  19 209]
  [  0  40 171]]]
